In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import minimize
import multiprocess as mp
from functools import partial
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def parallelize(data, func, num_of_processes=7):
    data_split = np.array_split(data, num_of_processes)
    pool = mp.Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [4]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    a = kappa*theta
    b = kappa+lambd   
    rspi = rho*sigma*phi*1j   
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )   
    g = (b-rspi+d)/(b-rspi-d)
    
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
    return exp1*term2*exp2

In [5]:
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    P, umax, N = 0, 100, 10000
    dphi=umax/N
    for i in range(1,N):
        phi = dphi * (2*i + 1)/2
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)
        
        P += dphi * numerator/denominator
        
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [6]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r, K):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    return numerator/denominator

In [7]:
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r, K)
    
    real_integral, err = np.real( quad(integrand, 0, 100, args=args) )
    
    return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

In [ ]:
# Parameters to test model
S0 = 3839.5 # initial asset price
K = 3891.098104 # strike
r = 0.050706-0.017072 # risk free rate
tau = 1.0 # time to maturity
heston_price( S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [9]:
df_full = pd.read_csv('../final_data/call_surfaces/2022-12-30.csv')

In [10]:
df = df_full.sample(10)

In [11]:
S0 = df['S_t'].values
r = df['r'].values - df['q'].values
K = df['K'].values
tau = df['tau'].values
P = df['call_price'].values

params = {"v0": {"x0": 0.05, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.12, "lbub": [-1,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

def obj(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    P_hat = np.fromiter((heston_price(S0[i], K[i], v0, kappa, theta, sigma, rho, lambd, tau[i], r[i]) for i in range(S0.shape[0])), dtype='float')
    return np.sqrt(np.average(np.square(P_hat - P)))

In [12]:
obj(x0)

3.8472750262381563

In [13]:
result = minimize(obj, x0, tol = 1e-1, method='SLSQP', options={'maxiter': 1e2 }, bounds=bnds)

In [14]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.4176178379924
       x: [ 4.778e-02  3.013e+00  4.778e-02  2.954e-01 -7.638e-01
            1.598e-01]
     nit: 1
     jac: [ 6.579e+02 -2.801e-01  7.383e+02  1.014e-01 -8.822e-01
           -1.258e+01]
    nfev: 10
    njev: 1

In [52]:
df_meta = pd.read_pickle('../mask/mask.pkl').dropna(axis=0)

In [72]:
arr_h = os.listdir('../VAE/results')

In [79]:
dates = np.char.replace(np.array(arr_h), '.csv', '')

In [95]:
arr_done = os.listdir('results_2')

In [97]:
dates_done = np.char.replace(np.array(arr_done), '.csv', '')

In [82]:
df_meta_sub = df_meta.loc[df_meta['date'].isin(dates)]

In [9]:
meta_test = df_meta.iloc[-1]

In [10]:
def read_df(meta):
    df_vol_surface = pd.read_csv(f"../final_data/call_surfaces_2/{meta['date']}.csv")
    return df_vol_surface

In [152]:
params = {"v0": {"x0": 0.05, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.12, "lbub": [-1,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

def calc_params(df_vol_surface, mask=None):
    
    if type(mask) != type(None):
        df_rel = df_vol_surface.iloc[mask]
        
    else:
        df_rel = df_vol_surface
        
    S0 = df_rel['S_t'].values
    r = df_rel['r'].values
    K = df_rel['K'].values
    tau = df_rel['tau'].values
    P = df_rel['call_price'].values
    
    def obj(x):
        v0, kappa, theta, sigma, rho, lambd = [param for param in x]
        P_hat = np.fromiter((heston_price(S0[i], K[i], v0, kappa, theta, sigma, rho, lambd, tau[i], r[i]) for i in range(S0.shape[0])), dtype='float')
        return np.average(np.square(P_hat - P))
    
    result = minimize(obj, x0, tol = 5e-2, method='SLSQP', options={'maxiter': 3e2 }, bounds=bnds)
        
    return (*result.x, result.status, result.fun)

In [155]:
df = read_df(meta_test)

In [ ]:
x = calc_params(df, mask=None)

In [168]:
def calc_prices(df, v0, kappa, theta, sigma, rho, lambd):
    S0 = df['S_t'].values
    r = df['r'].values - df['q'].values
    K = df['K'].values
    tau = df['tau'].values
    return np.fromiter((heston_price(S0[i], K[i], v0, kappa, theta, sigma, rho, lambd, tau[i], r[i]) for i in range(S0.shape[0])), dtype='float')

In [160]:
def full_process(meta):
    m_s = meta.copy()
    try:
        df = read_df(meta)
        df_new = df.copy()
        for m_n in [10, 100, 350, 400]:
            mask = np.random.choice(400, m_n,replace=False)
            if m_n == 400:
                result = calc_params(df)
            else:
                result = calc_params(df, mask=mask)

            m_s[f'{m_n}_v0'] = result[0]
            m_s[f'{m_n}_kappa'] = result[1]
            m_s[f'{m_n}_theta'] = result[2]
            m_s[f'{m_n}_sigma'] = result[3]
            m_s[f'{m_n}_rho'] = result[4]
            m_s[f'{m_n}_lambd'] = result[5]
            m_s[f'{m_n}_status'] = result[6]
            m_s[f'{m_n}_fun'] = result[7]

            df_new[f'{m_n}_heston_price'] = calc_prices(df, result[0], result[1], result[2], result[3], result[4], result[5])
            if not m_n == 400:
                df_new[f'{m_n}_mask'] = df_new.index.isin(mask)

        df_new['sigma'] = df['sigma']
        df_new.to_csv(f"results_3/{meta['date']}.csv", index=False)
        m_s['status'] = 'success'
            
    except Exception as e:
        print(e)
        m_s['status'] = 'error'
        
    return m_s

In [159]:
full_process(df_meta.loc[df_meta['date'] == '2022-11-01'].iloc[0])

date                                                2022-11-01
mask_10      [169, 392, 44, 347, 150, 200, 287, 346, 105, 288]
mask_100     [142, 30, 74, 73, 179, 121, 36, 21, 223, 135, ...
mask_350     [140, 37, 380, 134, 155, 194, 353, 14, 37, 117...
10_v0                                                 0.052395
10_kappa                                              2.596921
10_theta                                              0.039811
10_sigma                                               0.04518
10_rho                                               -0.073524
10_lambd                                              0.021505
10_status                                                    0
10_fun                                              282.759071
status                                                 success
Name: 4993, dtype: object

In [161]:
status_df = pd.read_pickle('status.pkl')

In [162]:
status_df = status_df.loc[pd.to_datetime(status_df['date']) >= '2013-01-01'][::-1]

In [163]:
status_df_sub = status_df.loc[df_meta['date'].isin(dates)]

In [164]:
status_df_sub_done = status_df_sub.loc[~status_df_sub['date'].isin(dates_done)].copy()

In [167]:
res = parallelize_on_rows(status_df_sub, full_process)

In [170]:
res.to_pickle('status_3.pkl')

In [32]:
import tqdm

In [ ]:
for i, status_df_sub in tqdm.tqdm(enumerate(np.array_split(status_df_sub, 25)), total=25, position=0, leave=True):
    res = parallelize_on_rows(status_df_sub, full_process)
    res.to_pickle(f'status/status_{i}.pkl')

In [535]:
df_meta.to_pickle('status.pkl')